# Topic modelling of news headlines for prediction of news category with 8 topics¶

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier on the combined feature 'title_description_text' feature and assume 7 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import pyLDAvis.lda_model

## Load data

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

In [6]:
# append words to stopwords that contain no information
stop_words += ['wegen', 'melden', 'meldet', 'können', 'könnte', 'könnten', 'update', 'neu', 'neue', 'neues', 'ohne', 'letzte', 'letzter',
                  'letztes', 'eins', 'zwei', 'drei', 'vier', 'fünf', 'sechs', 'sieben', 'acht', 'neun', 'zehn',
                   'gehen', 'geht', 'wollen', 'wollte', 'wollt', 'jahr', 'jahre', 'fordern', 'fordert',
                  'warnen', 'warnt', 'frühjahr', 'frühling', 'sommer', 'herbst', 'winter', 'erneut', 'deutlich', 'schwer', 'jahren', 'woche',
                  'wochen', 'monat', 'monate', 'tag', 'tage', 'stunden', 'stunde', 'minuten', 'minuten', 'ende', 'beenden', 'endet']

## Extract features from 'title_description_text' 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_df=0.95, min_df=3, stop_words=stop_words)

In [9]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title_description_text_cleaned'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [10]:
dtm

<67513x29998 sparse matrix of type '<class 'numpy.int64'>'
	with 1026097 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

In [12]:
lda = LatentDirichletAllocation(n_components=8, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=8, random_state=42)

## Analyse extracted features

In [13]:
len(cv.get_feature_names_out())

29998

In [14]:
cv.get_feature_names_out()

array(['000', '01', '034', ..., 'đoković', 'войны', 'украинцы'],
      dtype=object)

In [15]:
len(lda.components_)

8

In [16]:
lda.components_

array([[ 7.56164138,  0.12510745, 14.78257842, ...,  0.12516389,
         0.12500012,  0.12500005],
       [13.51931177,  0.12501205,  0.94854029, ...,  0.12584713,
         0.12500012,  0.12500005],
       [ 0.12534724,  1.12493878, 11.88443432, ...,  0.13075383,
         3.12499924,  0.12500004],
       ...,
       [10.54936001,  0.12500001, 29.10208387, ...,  3.11796871,
         0.12500008,  0.12500003],
       [ 0.12530522,  0.12500001, 20.20509504, ...,  0.1250001 ,
         0.12500011,  5.12499969],
       [ 6.7344875 ,  0.12500001,  0.25120873, ...,  0.12500011,
         0.12500012,  0.12500005]])

In [17]:
len(lda.components_[0])

29998

### Show most important words of first extracted topic

In [18]:
first_topic = lda.components_[0]

In [19]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([ 1589, 12143,  9271, ..., 25501, 19899, 21156], dtype=int64)

In [20]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [21]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

focus
online
moskau
un
russisch
china
krieg
ukraine
putin
russland


### Show most important words of first extracted topic

In [22]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['land', 'präsident', 'geben', 'kreml', 'westen', 'focus', 'online', 'moskau', 'un', 'russisch', 'china', 'krieg', 'ukraine', 'putin', 'russland']


Die TOP-15 Wörter für das Thema #1
['team', 'spiel', 'sieg', 'erster', 'partei', 'dfb', 'fc', 'bundesliga', 'gewinnen', 'deutsch', 'fußball', 'frau', 'bayern', 'wm', 'afd']


Die TOP-15 Wörter für das Thema #2
['groß', 'grüne', 'stehen', 'kanzler', 'lindner', 'gut', 'geben', 'wahl', 'deutsch', 'online', 'cdu', 'focus', 'spd', 'scholz', 'berlin']


Die TOP-15 Wörter für das Thema #3
['gipfel', 'g7', 'inflation', 'baerbock', 'deutsch', 'hoch', 'generation', 'zahl', 'china', 'steigen', 'geben', 'deutschland', 'us', 'pandemie', 'corona']


Die TOP-15 Wörter für das Thema #4
['verurteilen', 'regierung', 'weit', 'donald', 'iran', 'biden', 'gericht', 'ex', 'polizei', 'protest', 'mensch', 'usa', 'trump', 'präsident', 'us']


Die TOP-15 Wörter für das Thema #5
['mensch', 'groß', 'geld', 'million', 'bundesregierung

### Get topics with highest probability for news items in train data

In [23]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 8)

In [24]:
topic_results[0].round(5)

array([0.00544, 0.00544, 0.11746, 0.00544, 0.00544, 0.74665, 0.00544,
       0.1087 ])

In [25]:
topic_results[0].argmax()

5

In [26]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [27]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...,5
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...,0
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...,2
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...,6
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...,7


## Visualize model

In [28]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
5       1.676457  20.251356       1        1  17.160394
2      59.708809 -37.426018       2        1  14.942608
4      -7.054173 -60.902164       3        1  14.102786
6     -66.996574 -23.303980       4        1  12.324258
1     -76.050507  46.957352       5        1  10.692478
0      82.252655  29.598999       6        1  10.376446
7     -25.304579  96.439117       7        1  10.317649
3      45.198559  89.903450       8        1  10.083380, topic_info=           Term          Freq         Total Category  logprob  loglift
25501   ukraine  11798.000000  11798.000000  Default  30.0000  30.0000
7611         eu   4071.000000   4071.000000  Default  29.0000  29.0000
21156  russland   5986.000000   5986.000000  Default  28.0000  28.0000
14353     krieg   5947.000000   5947.000000  Default  27.0000  27.0000
5108     corona   2936.000000   2936.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
26189       usa    530.127959   2942.981253   Topic8  -5.4014   0.5802
24876    ticker    398.581577   1034.884534   Topic8  -5.6866   1.3401
3938    bleiben    449.180880   2183.508608   Topic8  -5.5671   0.7130
16451    mensch    401.027337   4178.486708   Topic8  -5.6805  -0.0494
10761       gut    367.618493   2735.963494   Topic8  -5.7674   0.2871

[587 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
83         1  0.990072              49
194        6  0.995748             abe
216        7  0.990585        abfeuern
627        1  0.000690             afd
627        5  0.998967             afd
...      ...       ...             ...
29792      4  0.563386       überblick
29792      5  0.071783       überblick
29792      7  0.094623       überblick
29792      8  0.066345       überblick
29934      3  0.991680  überschwemmung

[1332 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 5, 7, 2, 1, 8, 4])

## Save train set with topics and dump countvectorizer and model

In [29]:
#Save result to csv
train.to_csv('evaluation/train_lda_combined_8.csv')

In [30]:
# Dump countvectorizer
joblib.dump(cv, 'models/cv_combined.jl')

['models/cv_combined.jl']

In [31]:
# Dump LDA model
joblib.dump(lda, 'models/lda_model_8_topics_combined.jl')

['models/lda_model_8_topics_combined.jl']